In [1]:
#2차원구조 만들어서 만든 함수 불러와서 써보기

In [10]:
import numpy as np
from layers import Relu, SoftmaxWithLoss, Sigmoid, Affine
from collections import OrderedDict
from loss import cross_entropy_error
from activation import softmax

In [11]:
#이거 수정: 빠르게 나오게 하려고
class TwoLayerNet: 
    def __init__(self,input_size,hidden_size,output_size):
        self.W = {}
        self.W['W1'] = np.random.randn(input_size,hidden_size)
        self.W['b1'] = np.random.randn(hidden_size)
        self.W['W2'] = np.random.randn(hidden_size,output_size)
        self.W['b2'] = np.random.randn(output_size)
        self.loss_val = []
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.W['W1'],self.W['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.W['W2'],self.W['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
    
    def predict(self,x):
        for layer in self.layers.values(): #x업데이트
            x = layer.forward(x)
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        # loss = cross_entropy_error(y,t) #미분하려고 구한거(수치적) >> 해석적으로 쓰려고 class만듦 :forwardmax, backwardmax
        loss = self.lastLayer.forward(y,t)
        return loss

    def numerical_gradient(self,x,t):
        f = lambda W: self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(f, self.W['W1'])
        grads['b1'] = numerical_gradient(f, self.W['b1'])
        grads['W2'] = numerical_gradient(f, self.W['W2'])
        grads['b2'] = numerical_gradient(f, self.W['b2'])
        
        return grads
    
    def gradient(self,x,t): #backward시켜주는거 #맨 마지막loss부터 거꾸로 #grad(기울기)를 원래 값에서 빼주면 학습이 되는것이기 때문
        self.loss(x,t)
        dout = 1
        dout = self.lastLayer.backward(dout) #미분값
        layers = list(layers.values()) #끝까지 계산했다 다시 back해줘야해서 순서 바꾸기
        layers.reverse()
        for layer in layers:
            layer.backward(dout) 
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1)
        t = np.argmax(t,axis=1)
        acc = sum(y == t)/x.shape[0]
        return acc
    
    def fit(self,epochs,lr,x,t):
        for epoch in range(epochs):
            grads = self.gradient(x,t)
            for key in grads.keys():
                self.W[key] -= lr*grads[key]
            self.loss_val.append(self.loss(x,t))


In [12]:
#확인
x = np.random.randn(10,5)
t = np.random.randn(10,4)
input_size = x.shape[1]
hidden_size = 3
output_size = 4

W={}
W['W1'] = np.random.randn(input_size,hidden_size)
W['b1'] = np.random.randn(hidden_size)
W['W2'] = np.random.randn(hidden_size,output_size)
W['b2'] = np.random.randn(output_size)
layers = OrderedDict({}) #그냥 딕이 아닌 리스트로 만들어줘서 함수 쓰기 적합?
layers['Affine1'] = Affine(W['W1'],W['b1'])
layers['Relu1'] = Relu()
layers['Affine2'] = Affine(W['W2'],W['b2'])

In [5]:
layers #각 class확인가능

OrderedDict([('Affine1', <layers.Affine at 0x15f020f4df0>),
             ('Relu1', <layers.Relu at 0x15f020f41f0>),
             ('Affine2', <layers.Affine at 0x15f020f4970>)])

In [6]:
for layers in layers.values(): 
    print(layers)

In [17]:
for i in [layers.values()]: 
    print(i)

odict_values([<layers.Affine object at 0x0000015F04794970>, <layers.Relu object at 0x0000015F04794A30>, <layers.Affine object at 0x0000015F04794490>])


In [22]:
#Affine forward가 predict와 같음을 증명
def predict(self,x):
    w1,w2 = self.W['W1'],self.W['W2']
    b1,b2 = self.W['b1'],self.W['b2']
    a1 = np.dot(x,W1)+self.b1
    z1 = relu(a1)
    a2 = np.dot(z1,W2)+self.b2
    y = softmax(a2)